In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import pickle

In [2]:
def pickleIO(obj, src, op="r"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj

In [9]:
def get_news_summary(key_word):
    url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={key_word}'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}
    res = requests.get(url, headers= headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_list = soup.find_all('div', attrs={'class':'info_group'})

    news_url_list = [] # 네이버 뉴스 url 리스트
    new_article_list= [] # 네이버 뉴스 기사 통합 리스트
    new_article_head_list = [] 
    sec_article_list = []
    for i in range(9): # 최대 5개 정도만 추출
        news_article = news_list[i].find_all("a", attrs={"class":"info"})
        if len(news_article) > 1:
            news_nm_url = news_article[1].get('href')
            # print(news_nm_url)
            news_url_list.append(news_nm_url)
            res = requests.get(news_nm_url, headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            try:
                news_article_nm = soup.find_all('div', attrs = {'class': 'newsct_article _article_body'})
                aritle = soup.find(attrs={'class' : 'go_trans _article_content'}).text
                # print(i, '번쨰껄 가지고 옵니다. ')
                sec_article_list.append(aritle)
                news_article_head = soup.find( attrs = {'id': 'title_area'})
                news_article_text = news_article_nm[0].get_text()
                news_article_head_text = news_article_head.get_text()
                new_article_list.append(news_article_text)
                new_article_head_list.append(news_article_head_text)
            except:
                pass

    return pd.DataFrame({"head": new_article_head_list, "article": new_article_list, "sector": sec_article_list})

In [10]:
df_news = get_news_summary("삼성전자")
pickleIO(df_news, "./news_삼성전자.pkl", "w")

In [11]:
def get_review_from_11st(query):
    # Browser configuration
    # WARNiNG: chrome browser must be installed in local machine
    chromedriver_autoinstaller.install()
    service = Service(executable_path="chromedriver")
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    # Link to root URL
    url = f'https://search.11st.co.kr/Search.tmall?kwd={query}'
    driver.get(url); driver.implicitly_wait(5)
    # Extract product meta information
    prod_list = driver.find_elements(By.CSS_SELECTOR, "#layBodyWrap > div > div > div.l_search_content > div > section:nth-child(4) > ul > li:nth-child(n) > div > div:nth-child(2) > div.c_card_info_top > div.c_prd_name.c_prd_name_row_1")
    prod_title_list = [i.find_element(By.TAG_NAME, "strong").text for i in prod_list]
    link_list = [i.find_element(By.TAG_NAME, "a").get_attribute("href") for i in prod_list]
    # create dataframe for review data
    df_merge = []
    for link, title in zip(link_list, prod_title_list):
        # link to product page
        driver.get(link); driver.implicitly_wait(5)
        # switch to review iframe
        review_frame = driver.find_element(By.CSS_SELECTOR, "#ifrmReview")
        driver.switch_to.frame(review_frame)
        # extract review data
        reviews = [i.text for i in driver.find_elements(By.CSS_SELECTOR, "#review-list-page-area > ul > li:nth-child(n) > div.c_product_review_cont > div > div.cont_text_wrap > p.text-expanded")]
        grades = [i.text for i in driver.find_elements(By.CSS_SELECTOR, "#review-list-page-area > ul > li:nth-child(n) > div.c_product_review_cont > p > span > em")]
        review_output = {
            "prod_name": [],
            "review": [],
            "grade": [],
        }
        for review, grade in zip(reviews, grades):
            if len(review) > 0:
                review_output["prod_name"].append(title)
                review_output["review"].append(review)
                review_output["grade"].append(grade)
        df_merge.append(pd.DataFrame(review_output))
        time.sleep(1)
    # concatenate all reviews on products
    df_merge = pd.concat(df_merge).reset_index(drop=True)
    return df_merge

In [12]:
df_review_shop = get_review_from_11st("LG그램")
pickleIO(df_review_shop, "./review_lg그램.pkl", "w")

In [13]:
def get_content_review():
    # Browser configuration
    # WARNiNG: chrome browser must be installed in local machine
    chromedriver_autoinstaller.install()
    service = Service(executable_path="chromedriver")
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    # Link to root URL (네이버 영화 고요의 바다 리뷰 페이지)
    url = f'https://tv.naver.com/v/23742179#comment_focus'
    driver.get(url); driver.implicitly_wait(5)
    time.sleep(3)
    # stop video
    driver.find_element(By.CSS_SELECTOR, "video.webplayer-internal-video").click()
    # get reviews
    review_list = driver.find_elements(By.CSS_SELECTOR, "ul.u_cbox_list > li > div > div > div.u_cbox_text_wrap > span")
    review_list = [i.text for i in review_list]
    driver.close()
    return review_list

In [14]:
df_review_netflix = get_content_review()
df_review_netflix = pd.DataFrame(df_review_netflix)
df_review_netflix.columns = ["review"]
pickleIO(df_review_netflix, "./review_netflix.pkl", "w")